In [1]:
import pandas as pd
import os
import shutil
import librosa
import numpy as np
import soundfile as sf
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
import random
import tensorflow_io as tfio
import preprocessing as pr
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import argparse as ap
from glob import glob
import preprocessing as pr

31
Frame_length: 512
Frame_length: 512


In [2]:
saved_model_dir = f'./saved_models/last_model_used'

# Evaluation
filenames = glob('../../datasets/dsl_data/Test_Dataset_Truncated/*')
print("Files to be evaluated: ",len(filenames))

model = keras.models.load_model(saved_model_dir)
# predictions_single = model.predict(img)

Files to be evaluated:  1455


In [6]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()
tflite_models_dir = './tflite_models/'
if not os.path.exists(tflite_models_dir):
    os.makedirs(tflite_models_dir)
tflite_model_name = os.path.join(tflite_models_dir, 'last_model_savedtflite')
with open(tflite_model_name, 'wb') as fp:
    fp.write(tflite_model)
interpreter = tf.lite.Interpreter(model_path=tflite_model_name)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)

[{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([  1, 125,  31,   1]), 'shape_signature': array([ -1, 125,  31,   1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [17]:
with open("Evaluation_Dataset_Result.csv", "w") as file:
    file.write("Id,Predicted")
    #file.write("\n") 
    file.write("")
    for filename in filenames:
        identifier = filename.replace("\\", "/").split('/')[-1].split('.')[0]
        #filename = filename.split('/')[-1].split('.')[0]
        print(identifier)
        audio_binary = tf.io.read_file(filename)
        audio, sampling_rate = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=-1) #all our audio are mono, drop extra axis
        audio_padded = audio
        stft = tf.signal.stft(
            audio,
            frame_length=pr.frame_length,
            frame_step=pr.frame_step,
            fft_length=pr.frame_length
        )
        spectrogram = tf.abs(stft)
        mel_spectrogram = tf.matmul(spectrogram, pr.linear_to_mel_weight_matrix)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, -1)  # channel axis
        mfcss = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        
        mfcss = tf.expand_dims(mfcss, 0)
        prediction = model.predict(mfcss)
        
        prediction = np.argmax(prediction[0])
        prediction = pr.LABELS[prediction]
        
        print(prediction)
        file.write("\n{},{}".format(identifier,prediction))
        

0
1/1 [==============================] - 0s 141ms/step
decreasevolume
1
1/1 [==============================] - 0s 49ms/step
decreasevolume
10
1/1 [==============================] - 0s 114ms/step
decreasevolume
100
1/1 [==============================] - 0s 83ms/step
decreasevolume
1000
1/1 [==============================] - 0s 102ms/step
decreasevolume
1001
1/1 [==============================] - 0s 65ms/step
decreasevolume
1002
1/1 [==============================] - 0s 87ms/step
decreasevolume
1003
1/1 [==============================] - 0s 88ms/step
decreasevolume
1004
1/1 [==============================] - 0s 62ms/step
decreasevolume
1005
1/1 [==============================] - 0s 77ms/step
decreasevolume
1006
1/1 [==============================] - 0s 87ms/step
decreasevolume
1007
1/1 [==============================] - 0s 74ms/step
decreasevolume
1008
1/1 [==============================] - 0s 89ms/step
decreasevolume
1009
1/1 [==============================] - 0s 71ms/step
decreasevolum

KeyboardInterrupt: 

        interpreter.set_tensor(input_details[0]['index'], mfcss)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details[0]['index'])
        
        print(output)
        top_index = np.argmax(output[0])
        predicted_label = pr.LABELS[top_index]
        
        
        print(predicted_label)
        file.write("")